# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing
from msresist.clustering import MassSpecClustering
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv")

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

Drop NaNs:

In [4]:
# d = d.iloc[:8, :].dropna(axis=1)
# i = i.iloc[d.columns, :]

Reduce data set:

In [5]:
npept = 5000
d = d.iloc[:, :npept]
i = i.iloc[:npept, :]

In [6]:
d

0         1         2     3     4         5         6     7     \
C3N.01799     NaN       NaN  0.012034   NaN   NaN  0.477744  0.256791   NaN   
C3N.01799.N   NaN       NaN -0.630863   NaN   NaN -1.231926 -0.669631   NaN   
C3L.01890     NaN       NaN -1.159479   NaN   NaN -1.600599  0.088833   NaN   
C3L.01890.N   NaN       NaN -0.740044   NaN   NaN -1.349858 -0.226617   NaN   
C3N.00572     NaN       NaN  0.107228   NaN   NaN  0.388473  0.249418   NaN   
...           ...       ...       ...   ...   ...       ...       ...   ...   
C3N.01823.N   NaN -0.568015       NaN   NaN   NaN       NaN       NaN   NaN   
C3L.02549     NaN  0.254230       NaN   NaN   NaN       NaN       NaN   NaN   
C3L.02549.N   NaN -0.378522       NaN   NaN   NaN       NaN       NaN   NaN   
C3L.02365     NaN  0.987749       NaN   NaN   NaN       NaN       NaN   NaN   
C3L.02365.N   NaN -0.189486       NaN   NaN   NaN       NaN       NaN   NaN   

             8     9         10    11    12        13        14        15    \
C3N.01799     NaN   NaN  0.614297   NaN   NaN  0.814919       NaN -0.525557   
C3N.01799.N   NaN   NaN -1.184385   NaN   NaN -0.402629       NaN -0.147119   
C3L.01890     NaN   NaN  0.221363   NaN   NaN -2.372470       NaN  0.543993   
C3L.01890.N   NaN   NaN -0.674562   NaN   NaN -1.537937       NaN -0.099771   
C3N.00572     NaN   NaN  0.297154   NaN   NaN -0.142229       NaN -0.518668   
...           ...   ...       ...   ...   ...       ...       ...       ...   
C3N.01823.N   NaN   NaN -0.046686   NaN   NaN  0.261110 -0.272876       NaN   
C3L.02549     NaN   NaN  0.085890   NaN   NaN  1.157337  0.829829       NaN   
C3L.02549.N   NaN   NaN -0.519199   NaN   NaN  0.545468 -0.220462       NaN   
C3L.02365     NaN   NaN  1.033386   NaN   NaN  0.782913  1.228747       NaN   
C3L.02365.N   NaN   NaN -0.303490   NaN   NaN -0.105396 -0.139799       NaN   

                 16        17        18        19        20        21    \
C3N.01799         NaN  0.369009  0.834128  0.987832       NaN       NaN   
C3N.01799.N       NaN -0.648373 -1.235700 -0.589544       NaN       NaN   
C3L.01890         NaN -0.553806 -0.317504  0.753296       NaN       NaN   
C3L.01890.N       NaN -0.076537 -0.906582  0.334983       NaN       NaN   
C3N.00572         NaN  0.164563  0.133579  0.097229       NaN       NaN   
...               ...       ...       ...       ...       ...       ...   
C3N.01823.N -0.427656  0.316669 -0.456534 -0.593500  0.178943 -0.631947   
C3L.02549    0.012985  0.287304  0.637770  1.410095 -0.300406  1.961825   
C3L.02549.N -0.308487 -0.171400 -0.376306 -0.219179 -0.078853 -0.648981   
C3L.02365    0.246623  0.147777  0.946792  0.781463  0.262415  0.215821   
C3L.02365.N -0.336339  0.808027 -0.339930 -0.259864  0.195759 -0.383924   

                 22    23        24    25    26        27        28    29    \
C3N.01799    0.286759   NaN -0.988079   NaN   NaN  0.355716  0.351364   NaN   
C3N.01799.N -1.233980   NaN  0.574266   NaN   NaN -1.460670 -0.641403   NaN   
C3L.01890   -0.412355   NaN -1.795130   NaN   NaN  0.249927  0.215157   NaN   
C3L.01890.N -0.034192   NaN -1.162167   NaN   NaN -0.686302 -0.394540   NaN   
C3N.00572   -0.044064   NaN -1.323939   NaN   NaN  0.118925 -0.058038   NaN   
...               ...   ...       ...   ...   ...       ...       ...   ...   
C3N.01823.N       NaN   NaN  0.807877   NaN   NaN -0.347478 -0.519879   NaN   
C3L.02549         NaN   NaN -1.682482   NaN   NaN  0.648435  0.379704   NaN   
C3L.02549.N       NaN   NaN  1.827634   NaN   NaN -0.370233 -0.369936   NaN   
C3L.02365         NaN   NaN -1.818610   NaN   NaN  0.471396  0.708087   NaN   
C3L.02365.N       NaN   NaN -0.525439   NaN   NaN -0.151387  0.090106   NaN   

             30        31    32        33    34    35        36    37    38    \
C3N.01799     NaN  0.036711   NaN -0.304978   NaN   NaN  0.788038   NaN   NaN   
C3N.01799.N   NaN -1.347090   NaN  0.036237   NaN   NaN -1.591909   NaN   NaN   
C3L.01890     NaN  0.0

In [15]:
distance_method = "PAM250"
ncl = 9
SeqWeight = 2000000000000

MSC = MassSpecClustering(i, ncl, GMMweight=SeqWeight, distance_method=distance_method, n_runs=1).fit(d, "NA")

initialized
N_ITER:  0
SeqWins:  4810 DataWins:  0 BothWin  190 MixWins:  0
CLUSTER SIZES:  [224, 93, 8, 230, 2145, 751, 1259, 129, 161]
-75729269195.47334
N_ITER:  1


AssertionError: final score is either     NaN or -Inf, motif = AAAAASGGGVS, gmmp = [[ 0.  0. nan ...  0.  0.  0.]
 [ 0.  0. nan ...  0.  0.  0.]
 [ 0.  0. nan ...  0.  0.  0.]
 ...
 [ 0.  0. nan ...  0.  0.  0.]
 [ 0.  0. nan ...  0.  0.  0.]
 [ 0.  0. nan ...  0.  0.  0.]], nonzeros = 5000

In [ ]:
centers = MSC.transform(d)

In [ ]:
# 1 - np.exp(-0.08040164617182279)